In [42]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit
import random as rd
import math
from qiskit import algorithms
from qiskit.circuit import Parameter
from qiskit.opflow import X,Z,I,CX,CZ
classical_optimizer = algorithms.optimizers.SPSA #choix de l'optimiseur avant la routine VQE 

 étape 1 : Initialisation, on se propose de créer un ansatz variationnel pour N = 10 spins selon la méthode suivante

In [3]:
N=10
# initialisation
def ansatz(N): 
    qc = QuantumCircuit(N)
    for k in range(N):
        thet = rd.random()*2*math.pi
        qc.rx(thet,k)
    return(qc)
Ansatz = ansatz(N)

theta = Parameter('t_' + str(0))

qc = Ansatz

qc.rx(theta,1)

print("notre ansatz est le suivant")

Ansatz.draw()


notre ansatz est le suivant


┌────────────┐           
q_0: ┤ Rx(4.8955) ├───────────
     ├────────────┤┌─────────┐
q_1: ┤ Rx(2.6297) ├┤ Rx(t_0) ├
     ├────────────┤└─────────┘
q_2: ┤ Rx(4.1583) ├───────────
     ├────────────┤           
q_3: ┤ Rx(1.6297) ├───────────
     ├────────────┤           
q_4: ┤ Rx(4.4743) ├───────────
     ├────────────┤           
q_5: ┤ Rx(1.7795) ├───────────
     ├────────────┤           
q_6: ┤ Rx(3.0723) ├───────────
     ├────────────┤           
q_7: ┤ Rx(1.6437) ├───────────
     ├────────────┤           
q_8: ┤ Rx(4.9379) ├───────────
     ├────────────┤           
q_9: ┤ Rx(5.5158) ├───────────
     └────────────┘

In [4]:
# notre pool d'opérateurs est initialisé selon la méthode suivante 
OPS = []    
for k in range(N): 
    OPS.append( [True,k])
for k in range(N): 
    for j in range(N):
        OPS.append([False,[k,j]])


choisissons n opérateurs avec n = N/2

In [5]:
#select operators from pool

n = N//2
L_c = []
OPS1 = OPS.copy()
for k in range(n):
    j = int(rd.random()*len(OPS1))
    L_c.append(OPS1[j])
    del OPS1[j]
print(L_c)


[[False, [9, 3]], [True, 7], [True, 2], [True, 3], [False, [5, 9]]]


In [43]:
#transfer gates to other qbits

def trans(gate, n, i):
    #Envoie le "gate" de OperatorBase sur le qbit i, dont n est le nb total
    k = gate.num_qubits
    
    return(I.tensorpower(max(n-i-k, 0)) ^ gate ^ I.tensorpower(i))

print(trans(X,10,5))

IIIIXIIIII


In [7]:
#Compute the Hamiltonian
from qiskit.opflow import *
##Phase de test
J = 2
h = 3

"Hamiltonien pour 2 qbits:"

H = - J * CX.compose(Z.compose(CX)) - h * (X ^ I)

print(H)

ComposedOp(H).to_circuit().draw()
##Fin de la phase de test

SummedOp([
  -2.0 *           ┌────────────┐     
  q_0: ──■──┤0           ├──■──
       ┌─┴─┐│  Pauli(ZI) │┌─┴─┐
  q_1: ┤ X ├┤1           ├┤ X ├
       └───┘└────────────┘└───┘,
  -3.0 * XI
])


┌────────────┐     ┌────────────┐     
q_0: ┤0           ├──■──┤0           ├──■──
     │  Pauli(XI) │┌─┴─┐│  Pauli(ZI) │┌─┴─┐
q_1: ┤1           ├┤ X ├┤1           ├┤ X ├
     └────────────┘└───┘└────────────┘└───┘

In [9]:
def hi(N, i):
    #n = nombre de qubits
    #i = indice
    if i >= N-1:
        return('Error Hi: The number of qubits is too small')
    H = (- J * CX.compose(Z.compose(CX))) - (h * (X ^ I))
    if i != 0:
        H = I.tensorpower(i) ^ H
    if i + 1 == N-1:
        return(H)
    else:
        H = H ^ I.tensorpower(N-i-2)
    return(H)

c = hi(5, 2)
#print(c)


hamil = hi(N, 0)
for i in range(1, N-1):
    hamil += hi(N, i)


print(hamil)

SummedOp([
  TensoredOp([
    SummedOp([
      -2.0 *           ┌────────────┐     
      q_0: ──■──┤0           ├──■──
           ┌─┴─┐│  Pauli(ZI) │┌─┴─┐
      q_1: ┤ X ├┤1           ├┤ X ├
           └───┘└────────────┘└───┘,
      -3.0 * XI
    ]),
    IIIIIIII
  ]),
  TensoredOp([
    I,
    SummedOp([
      -2.0 *           ┌────────────┐     
      q_0: ──■──┤0           ├──■──
           ┌─┴─┐│  Pauli(ZI) │┌─┴─┐
      q_1: ┤ X ├┤1           ├┤ X ├
           └───┘└────────────┘└───┘,
      -3.0 * XI
    ]),
    IIIIIII
  ]),
  TensoredOp([
    II,
    SummedOp([
      -2.0 *           ┌────────────┐     
      q_0: ──■──┤0           ├──■──
           ┌─┴─┐│  Pauli(ZI) │┌─┴─┐
      q_1: ┤ X ├┤1           ├┤ X ├
           └───┘└────────────┘└───┘,
      -3.0 * XI
    ]),
    IIIIII
  ]),
  TensoredOp([
    III,
    SummedOp([
      -2.0 *           ┌────────────┐     
      q_0: ──■──┤0           ├──■──
           ┌─┴─┐│  Pauli(ZI) │┌─┴─┐
      q_1: ┤ X ├┤1           ├┤ X ├
    

In [15]:
#compute commutators for each operators chosen

"trucs à changer : Mettre au choix X ou Y ou Z quand True"

def perm(k):
    "ici k est une liste de deux entiers distincts compris entre 0 et N-1"
    res = [i for i in range(N)] #on créée l'ordre identité
    p0 = k[0]
    p1 = k[1]
    res[p1], res[p0 + 1] = p0 + 1, p1
    return(res)



def gradient(ansatz, hamil):
    list = []
    for k in L_c : 
        if k[0] : 
            operator = trans(X,N,k[1])
        
            Com = commutator(hamil,operator)
            list.append([operator,Avg])
        
        else : 
            operator = trans(CX.permute([1, 0]), N, k[1][0]).permute(perm(k[1])) 
        
            Com = commutator(hamil,operator)
        
            list.append([operator,Avg])
    return(list)
gradient(ansatz, hamil)

IndexError: list assignment index out of range

In [125]:
l_op = gradient(ansatz,hamil)
l_v = []
for operator in l_op: 
    vqe = algorithms.VQE(Ansatz,classical_optimizer)   
    Avg = vqe.get_energy_evaluation(Com, False)
    Ansatz.parameters

ParameterView([Parameter(t_0)])

In [126]:
t = len(Ansatz.parameters)

In [10]:
Avg(np.zeros(1))

NameError: name 'Avg' is not defined

In [ ]:
vqe = algorithms.VQE(ansatz,classical_optimizer, param)
Avg = vqe.get_energy_evaluation(Com, False)
vqe = algorithms.VQE(ansatz,classical_optimizer)
Avg = vqe.get_energy_evaluation(Com, False)

In [ ]:
X_4 = trans(X,10,4)
Z_4 = trans(Z,10,4)

com = 1/2 *commutator(X_4,Z_4)

print(com)
com.is_hermitian

In [88]:
qc = ansatz(10)

theta = Parameter('t_0')

qc.rx(theta,3)

print(qc)



      ┌────────────┐           
q_0: ─┤ Rx(3.4031) ├───────────
      ├────────────┤           
q_1: ─┤ Rx(5.1333) ├───────────
      ├────────────┤           
q_2: ─┤ Rx(1.0726) ├───────────
      ├───────────┬┘┌─────────┐
q_3: ─┤ Rx(4.261) ├─┤ Rx(t_0) ├
      ├───────────┴┐└─────────┘
q_4: ─┤ Rx(0.7839) ├───────────
      ├────────────┤           
q_5: ─┤ Rx(1.4193) ├───────────
      ├────────────┤           
q_6: ─┤ Rx(1.1311) ├───────────
     ┌┴────────────┤           
q_7: ┤ Rx(0.25193) ├───────────
     └┬────────────┤           
q_8: ─┤ Rx(5.7028) ├───────────
     ┌┴────────────┤           
q_9: ┤ Rx(0.87312) ├───────────
     └─────────────┘           


In [89]:
vqe = algorithms.VQE(qc,classical_optimizer)

Avg = vqe.get_energy_evaluation(com, False)


In [90]:
type(Avg)

function

In [91]:
Avg(np.array([math.pi]))

AttributeError: 'NoneType' object has no attribute 'convert'

In [65]:
Avg(np.array([0,0.1]))

AttributeError: 'NoneType' object has no attribute 'convert'

In [72]:
qc.append(com, [0,9])

KeyboardInterrupt: 

In [76]:
qc.append(X_4,[k for k in range(0,10)])

In [82]:
print(qc)
qc.n
print(len(qc))

     ┌─────────────┐           ┌────────────────────┐
q_0: ┤ Rx(0.77299) ├───────────┤0                   ├
     └┬────────────┤           │                    │
q_1: ─┤ Rx(6.1715) ├───────────┤1                   ├
      ├───────────┬┘           │                    │
q_2: ─┤ Rx(6.073) ├────────────┤2                   ├
      ├───────────┤ ┌─────────┐│                    │
q_3: ─┤ Rx(6.278) ├─┤ Rx(t_0) ├┤3                   ├
      ├───────────┴┐└─────────┘│                    │
q_4: ─┤ Rx(3.9971) ├───────────┤4                   ├
      ├────────────┤           │  Pauli(IIIIIXIIII) │
q_5: ─┤ Rx(2.6001) ├───────────┤5                   ├
      ├────────────┤           │                    │
q_6: ─┤ Rx(1.6499) ├───────────┤6                   ├
      ├────────────┤           │                    │
q_7: ─┤ Rx(1.1181) ├───────────┤7                   ├
      ├────────────┤           │                    │
q_8: ─┤ Rx(2.9978) ├───────────┤8                   ├
      ├────────────┤        

In [87]:
qc_2 = ansatz(10)
print(len(qc_2))
print(ansatz(10))
qc_2.measure(4)

10
      ┌────────────┐ 
q_0: ─┤ Rx(1.4302) ├─
     ┌┴────────────┤ 
q_1: ┤ Rx(0.23875) ├─
     ├─────────────┤ 
q_2: ┤ Rx(0.12784) ├─
     └┬────────────┤ 
q_3: ─┤ Rx(1.0363) ├─
      ├────────────┤ 
q_4: ─┤ Rx(5.5849) ├─
      ├────────────┤ 
q_5: ─┤ Rx(5.5022) ├─
      ├────────────┤ 
q_6: ─┤ Rx(1.1841) ├─
      ├────────────┤ 
q_7: ─┤ Rx(4.7767) ├─
      ├────────────┤ 
q_8: ─┤ Rx(4.8238) ├─
     ┌┴────────────┴┐
q_9: ┤ Rx(0.033717) ├
     └──────────────┘


TypeError: measure() missing 1 required positional argument: 'cbit'

In [ ]:
def Energy(qc,H):
    qc.append(H, [k for k in range(len(qc))])
    